In [1]:
import torch
from ultralytics import YOLO 
import pandas as pd
import yaml
from symbolic_components.detector import Detector
from symbolic_components.positioner import Positioner
import clingo

ModuleNotFoundError: No module named 'symbolic_components'

In [ ]:
model = YOLO('/Users/maximvandecasteele/PycharmProjects/NeurASP/Object_detector/models/Mario_OD_vanilla_best.pt')

In [ ]:
class Solver:
    def __init__(self):
        super().__init__()
        self.atoms = []

    def solve(self, positions, show, locations):

        control = clingo.Control(message_limit=0)
        control.configuration.solve.models = 0

        # add asp
        control.add("base", [], positions)
        control.add("base", [], locations)
        control.add("base", [], show)

        control.ground([("base", [])])

        handle = control.solve(on_model=self.on_model)

        if handle.satisfiable:
            return self.atoms

        return None

    def on_model(self, model):
        """
        This is the observer callback for the clingo Control object after solving. It is done in a separate thread,
        which is why we use a new instantiation of the Solver class, otherwise its state is not thread safe
        :param model:
        """
        # print("Found solution:", model)
        symbols = model.symbols(shown=True)
        for symbol in symbols:
            # print(symbol)
            self.atoms.append(symbol)


In [ ]:
if torch.backends.mps.is_available():
    # mps_device = torch.device(self.device)
    print("Using mps device.")
    device = 'mps'
elif torch.cuda.is_available():
    device_name = torch.cuda.get_device_name(1)
    print("Using CUDA device:", device_name)
    device = 'cuda:1'
else:
    print("CUDA is not available")
    device = 'cpu'

In [2]:
if torch.backends.mps.is_available():
    config = {
            "detector_model_path": '/Users/maximvandecasteele/PycharmProjects/NeurASP/Object_detector/models/YOLOv8-Mario-lvl1-3/weights/best.pt',
            "detector_label_path": '/Users/maximvandecasteele/PycharmProjects/NeurASP/Object_detector/models/data.yaml',
            "positions_asp": '/Users/maximvandecasteele/PycharmProjects/NeurASP/RL/asp/positions.lp',
            "show_asp": '/Users/maximvandecasteele/PycharmProjects/NeurASP/RL/asp/show.lp',
            "constraints": '/Users/maximvandecasteele/PycharmProjects/NeurASP/RL/asp/game_rules.lp',
            "show_actions": '/Users/maximvandecasteele/PycharmProjects/NeurASP/RL/asp/show_constraints.lp'
        }
elif torch.cuda.is_available():
    config = {
            "detector_model_path": '/home/stefaan/local/python/NeurASP/Object_detector/models/YOLOv8-Mario-lvl1-3/weights/best.pt',
            "detector_label_path": '/home/stefaan/local/python/NeurASP/Object_detector/models/data.yaml',
            "positions_asp": '/home/stefaan/local/python/NeurASP/RL/asp/positions.lp',
            "show_asp": '/home/stefaan/local/python/NeurASP/RL/asp/show.lp',
            "constraints": '/home/stefaan/local/python/NeurASP/RL/asp/game_rules.lp',
            "show_actions" : '/home/stefaan/local/python/NeurASP/RL/asp/show_constraints.lp',
        }

In [3]:
f = open(config["constraints"])
constraints = f.read()
f.close()

f = open(config["show_actions"])
show = f.read()
f.close()

In [4]:
detector = Detector(config)
positioner = Positioner(config)

NameError: name 'Detector' is not defined

In [136]:
observation = '/Users/maximvandecasteele/PycharmProjects/NeurASP/RL/frame_constraint_checking/SuperMarioBros-1-1-v0img_SuperMarioBros-1-1-v0_2024-01-19-15_21_26.png'

In [137]:
positions = detector.detect(observation)
symbols = positioner.position(positions)

In [138]:
current_facts = ' '.join(symbols)

In [139]:
clingo_symbols = Solver().solve(constraints, show, current_facts)

In [140]:
clingo_symbols

[Function('action', [Number(4)], True), Function('action', [Number(2)], True)]

In [141]:
current_facts

'cell(13,0,4). cell(12,5,1). cell(12,7,3). cell(9,2,2). cell(9,8,2). cell(9,9,2). cell(5,0,2). cell(5,1,2). cell(13,1,2). cell(13,2,2). cell(13,3,2). cell(13,4,2). cell(13,5,2). cell(13,6,2). cell(13,7,2). cell(13,8,2). cell(13,9,2). cell(13,10,2). cell(13,11,2). cell(13,12,2). cell(13,13,2). cell(13,14,2). cell(13,15,2). cell(9,14,2). cell(5,2,2).'